In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List
from os.path import isfile 

In [173]:
class Transaction():
    def __init__(self) -> None:
        self.files = ["usage.xlsx","subscription.xlsx","development.xlsx","maint.xlsx","customization.xlsx"]
        self.SUBSCRIPTION_STRING = "Subsciption Fees"
        self.USAGE_STRING = "Usage fees"
        self.DEVELOPMENT_STRING = "Development fees"
        self.MAINT_STRING = "Recurring maint fees"
        self.CUSTOMISATION_STRING = "Customization"
    
    def get_dataframe(self, filename:str) -> pd.DataFrame:
        df = pd.read_excel(io=filename)
        df = df.drop(df.columns[[i for i in range(1,6)] + [8]], axis=1)
        df.columns = ['name', 'debit', 'credit']
        return df
    
    def get_transaction(self,filename:str) -> List:
        if not isfile(filename):
            return
        transactions:List = []
        entity:str = ""
        credit:float = 0
        debit:float = 0
        account_type:str = self.get_account_type(filename)
        df = self.get_dataframe(filename)
        
        for index, row in df.iterrows():
            if pd.isnull(row[0]):
                credit:float = 0
                debit:float = 0
                continue
            
            if type(row[0]) == datetime:
                credit += row["credit"]
                debit += row["debit"]
                continue
            
            string:str = row[0]
            if "Total" in string:
                string = string.replace("Total", "").strip()
                # Reset all counters and prepare for next entity
                transactions.append([f"\"{string}\"", account_type, credit-debit, credit, debit])
                credit:float = 0
                debit:float = 0
                continue
        return transactions

    def get_account_type(self,filename:str) -> str:
        if filename == "usage.xlsx":
            return self.USAGE_STRING
        elif filename == "subscription.xlsx":
            return  self.SUBSCRIPTION_STRING
        elif filename == "development.xlsx":
            return  self.DEVELOPMENT_STRING
        elif filename == "maint.xlsx":
            return  self.MAINT_STRING
        elif filename == "customization.xlsx":
            return  self.CUSTOMISATION_STRING
        return ""

    def get_transactions(self):
        transactions:List = []
        for file in self.files:
            # store transactions
            try:
                print(file)
                transactions.extend(self.get_transaction(file))
            except:
                pass
        self.save_as_csv(transactions)

    def save_as_csv(self,list_item:List) -> None:
        np.savetxt("compiled.csv", 
           list_item,
           delimiter =", ", 
           fmt ='%s')
    
    def compile_transactions(self):
        self.get_transactions()

In [174]:
t = Transaction()
t. compile_transactions()

usage.xlsx
subscription.xlsx
development.xlsx
maint.xlsx
customization.xlsx


In [4]:
filename = "all.xlsx"
df = pd.read_excel(io=filename)
df = df.drop(df.columns[[i for i in range(1,6)]], axis=1)
df.columns = ['name', 'debit', 'credit','account']

In [6]:
df.head(20)

,name,debit,credit,account
0,Wiz Holdings Pte. Ltd.,NaN,NaN,NaN
1,For the period 1 April 2023 to 30 April 2023,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,Date,Debit (SGD),Credit (SGD),Account
4,2023-04-25 00:00:00,14834.53,0,Sales - Subscription Fees
5,2023-04-25 00:00:00,15045.84,0,Sales - Development Fee
6,2023-04-25 00:00:00,22365.59,0,Sales - Subscription Fees
7,2023-04-26 00:00:00,0,178980.54,Sales - Usage Fees
8,2023-04-30 00:00:00,0,1666.67,Sales - Subscription Fees
9,2023-04-30 00:00:00,0,294.07,Sales - Subscription Fees


In [9]:
class Transaction():
    def __init__(self) -> None:
        self.files = "all.xlsx"
        self.SUBSCRIPTION_STRING = "Subsciption Fees"
        self.USAGE_STRING = "Usage fees"
        self.DEVELOPMENT_STRING = "Development fees"
        self.MAINT_STRING = "Recurring maint fees"
        self.CUSTOMISATION_STRING = "Customization"
    
    def get_dataframe(self, filename:str) -> pd.DataFrame:
        df = pd.read_excel(io=filename)
        df = df.drop(df.columns[[i for i in range(1,6)]], axis=1)
        df.columns = ['name', 'debit', 'credit','account']
        return df
    
    def create_record_holder(self) -> dict:
        revenue = dict()
        revenue_type = ["Sales - Development Fee","Sales - Subscription Fees","Sales - Usage Fees","Sales - Customization / Adhoc"]
        for item in revenue_type:
            revenue[item] = {"credit": 0, "debit":0}
        return revenue
    
    def record_to_transaction_entry(self, record:dict):
        pass
    
    def get_transaction(self,filename:str) -> List:
        if not isfile(filename):
            return
        transactions:List = []
        entity:str = ""
        credit:float = 0
        debit:float = 0
        df = self.get_dataframe(filename)
        
        for index, row in df.iterrows():
            if pd.isnull(row["name"]):
                record:dict = self.create_record_holder()
                continue
            
            if type(row["name"]) == datetime:
                # entry in the company
                # update the credit and debit for the account type
                record[row['account']]["credit"] += row["credit"]
                record[row['account']]["debit"] += row["debit"]
                continue
            
            string:str = row["name"]
            if "Total" in string:
                string = string.replace("Total", "").strip()
                if len(string) <= 0:
                    # reset record entry
                    record:dict = self.create_record_holder()
                    continue
                # Reset all counters and prepare for next entity
                for revenue_type in record:
                    transactions.append([f"\"{string}\"",
                                         revenue_type,
                                         record[revenue_type]["credit"]-record[revenue_type]["debit"],
                                         record[revenue_type]["credit"],
                                         record[revenue_type]["debit"]])
                # reset record entry
                record:dict = self.create_record_holder()
                continue
        return transactions

    def save_as_csv(self,list_item:List) -> None:
        np.savetxt("compiled.csv", 
           list_item,
           delimiter =", ", 
           fmt ='%s')
    
    def compile_transactions(self):
        transactions = self.get_transaction(self.files)
        self.save_as_csv(transactions)

In [10]:
t = Transaction()
t. compile_transactions()